# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [69]:
import numpy as np
import pandas as po
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [7]:
stocks = po.read_csv('sp_500_stocks.csv')

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [9]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [15]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'AMS/TOSCALTKS )D BG NQREELGEL(AAN', 'calculationPrice': 'close', 'open': 134, 'openTime': 1654008234689, 'openSource': 'fcoliifa', 'close': 131.927, 'closeTime': 1623491979663, 'closeSource': 'olfcifia', 'high': 129.5, 'highTime': 1682101310156, 'highSource': 'ur5p a1 elmnddt eiceyie', 'low': 128.51, 'lowTime': 1619251204460, 'lowSource': 'dimeyane tdui1elr ec 5p', 'latestPrice': 127.899, 'latestSource': 'Close', 'latestTime': 'December 18, 2020', 'latestUpdate': 1665368039994, 'latestVolume': 194018302, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 129.51, 'delayedPriceTime': 1663738179035, 'oddLotDelayedPrice': 130.06, 'oddLotDelayedPriceTime': 1675384595524, 'extendedPrice': 129.4, 'extendedChange': 0.652, 'extendedChangePercent': 0.00517, 'extendedPriceTime': 1681605829304, 'previousClose': 128.7, 'previousVolume': 96747166, 'change': -2.118, 'changePercent': -0.01666, 'v

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [17]:
price = data['latestPrice']
marketCap = data['marketCap']
print(marketCap)

2205840264087


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [18]:
myc = ['Ticker', 'Price/Stock','Market-Cap','Number to buy']
fdf = po.DataFrame(columns = myc)
fdf

,Ticker,Price/Stock,Market-Cap,Number to buy


In [23]:
fdf.append(
    po.Series(
    [
        symbol,
        price,
        marketCap,
        'N/A'
    ],
    index = myc),
    ignore_index=True
)

,Ticker,Price/Stock,Market-Cap,Number to buy
0,AAPL,127.899,2205840264087,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [31]:
fdf = po.DataFrame(columns = myc)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    fdf = fdf.append(
    po.Series(
    [
         stock,
        data['latestPrice'],
        data['marketCap'],
        'N/A'
    ],
    index = myc),
    ignore_index = True
    )

In [33]:
fdf

,Ticker,Price/Stock,Market-Cap,Number to buy
0,A,122.092,36179785756,N/A
1,AAL,16.010,9785760334,N/A
2,AAP,165.066,11214718482,N/A
3,AAPL,128.640,2139128668931,N/A
4,ABBV,102.330,181496722458,N/A
5,ABC,99.720,20489610129,N/A
6,ABMD,288.800,13230997597,N/A
7,ABT,107.790,192671286162,N/A
8,ACN,270.390,179079987621,N/A
9,ADBE,504.400,239849236310,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [35]:
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

In [56]:
symbolGroups = list(chunks(stocks['Ticker'],100))
symbolStrings = []
for i in range(0,len(symbolGroups)):
    #print(symbolGroups[i])
    symbolStrings.append(','.join(symbolGroups[i]))
fdf = po.DataFrame(columns = myc)
for symbolString in symbolStrings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbolString}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    #print(data['AAPL']['quote']['latestPrice'])
    for symbol in symbolString.split(','):
        fdf = fdf.append(
            po.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
                index = myc),
            ignore_index=True
        )
fdf

,Ticker,Price/Stock,Market-Cap,Number to buy
0,A,121.950,37862757755,N/A
1,AAL,16.420,9889359677,N/A
2,AAP,164.170,11300412208,N/A
3,AAPL,127.186,2195774968389,N/A
4,ABBV,104.740,184022852546,N/A
5,ABC,99.500,20481767208,N/A
6,ABMD,289.410,13474233002,N/A
7,ABT,108.490,197213291415,N/A
8,ACN,262.126,174057734878,N/A
9,ADBE,506.220,244997282607,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [73]:
ps = input('Enter the value of your portfolio: ')

try:
    val = float(ps)
    print(val)
except ValueError:
    print("That's not a number! \nPlease try again: ")
    ps = input('Enter the value of your portfolio: ')
    val = float(ps)

Enter the value of your portfolio: 10000000
10000000.0


In [83]:
positionSize = val/len(fdf.index)
#numAshares = positionSize/500
#print(math.floor(numAshares))
for i in range(0,len(fdf.index)):
    fdf.loc[i,'Number to buy'] = math.floor(positionSize/(fdf.loc[i,'Price/Stock']))
fdf

,Ticker,Price/Stock,Market-Cap,Number to buy
0,A,121.950,37862757755,162
1,AAL,16.420,9889359677,1205
2,AAP,164.170,11300412208,120
3,AAPL,127.186,2195774968389,155
4,ABBV,104.740,184022852546,189
5,ABC,99.500,20481767208,199
6,ABMD,289.410,13474233002,68
7,ABT,108.490,197213291415,182
8,ACN,262.126,174057734878,75
9,ADBE,506.220,244997282607,39


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [84]:
write = po.ExcelWriter('recommended trades.xlsx',engine = 'xlsxwriter')
fdf.to_excel(writer, 'Recommended Trades', index = False)


### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
integer_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: